In [ ]:
from os import environ

input_folder = environ.get(
    'CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get(
    'CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output')
log_folder = environ.get(
    'CROSSCOMPUTE_LOG_FOLDER', 'tests/standard/log')
debug_folder = environ.get(
    'CROSSCOMPUTE_DEBUG_FOLDER', 'tests/standard/debug')

In [ ]:
import json
from os.path import join

settings_path = join(input_folder, 'settings.json')
d = json.load(open(settings_path, 'rt'))
survey_code = d['survey-code']
survey_code

In [ ]:
electricity_network_path = join(input_folder, 'electricity-network.json')
electricity_network_dictionary = json.load(open(electricity_network_path, 'rt'))
electricity_network_dictionary

In [ ]:
from invisibleroads_macros_disk import make_folder
from invisibleroads_macros_log import get_timestamp
from os.path import expanduser

storage_folder = make_folder(expanduser('~/Experiments/survey-infrastructure'))
survey_folder = make_folder(join(storage_folder, survey_code, get_timestamp()))
survey_folder

In [ ]:
import shutil
shutil.copy(electricity_network_path, join(output_folder, 'electricity-network.json'))
shutil.copy(electricity_network_path, join(survey_folder, 'electricity-network.json'))

In [ ]:
from collections import defaultdict

asset_count_by_type_code = defaultdict(int)
asset_by_id = electricity_network_dictionary['assets']['assetById']
for asset_id, asset_dictionary in asset_by_id.items():
    asset_type_code = asset_dictionary['typeCode']
    asset_count_by_type_code[asset_type_code] += 1

asset_count_by_type_code

In [ ]:
import pandas as pd
t = pd.DataFrame(asset_count_by_type_code.items(), columns=['type', 'count'])
t

In [ ]:
t = t.set_index('type').rename({
    'g': 'generator',
    't': 'transformer',
    'q': 'power quality',
    'm': 'meter',
    'l': 'line',
    'x': 'switch',
})

In [ ]:
t.to_csv(join(output_folder, 'statistics.csv'))

In [ ]:
asset_features = electricity_network_dictionary['assets']['assetsGeoJson']['features']
asset_features_by_type_code = defaultdict(list)
for asset_type_code in ['g', 'l', 'm']:
    for asset_feature in asset_features:
        if asset_feature['properties']['typeCode'] == asset_type_code:
            asset_features_by_type_code[asset_type_code].append(asset_feature)
# dict(asset_features_by_type_code)

In [ ]:
import geojson

def save_map(target_path, features):
    geojson.dump({'type': 'FeatureCollection', 'features': features}, target_path)

In [ ]:
save_map(open(join(output_folder, 'electricity-generators.geojson'), 'wt'), asset_features_by_type_code['g'])
save_map(open(join(output_folder, 'electricity-lines.geojson'), 'wt'), asset_features_by_type_code['l'])
save_map(open(join(output_folder, 'electricity-meters.geojson'), 'wt'), asset_features_by_type_code['m'])